### app.py

In [1]:
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup
from io import StringIO
import re
from pprint import pprint

In [2]:
# Fetch Disease-symptoms knowledge database
url = 'https://people.dbmi.columbia.edu/~friedma/Projects/DiseaseSymptomKB/index.html'

# Request page content
headers = {'User-Agent': 'Mozilla/5.0'}
response = requests.get(url, headers=headers)

if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    tables = soup.find_all('table', {'class': 'MsoTableWeb3'})

    if len(tables) > 0:
        table = tables[0]
        rows = table.find_all('tr')

        # Extract header from the first row
        headers = [th.get_text(strip=True) for th in rows[0].find_all(['th', 'td'])]

        # Extract all subsequent rows into a list of lists
        data = []
        for row in rows[1:]:
            cols = row.find_all('td')
            cols = [col.get_text(strip=True) for col in cols]
            if cols:
                data.append(cols)

        # Create DataFrame
        df = pd.DataFrame(data, columns=headers)

        # Print preview
        print("Table scraped successfully")


        # Now df holds your table as a DataFrame and you can continue using it
    else:
        print("No table with class 'MsoTableWeb3' found.")
else:
    print(f"Failed to fetch the page. Status code: {response.status_code}")
df

Table scraped successfully


,Disease,Count of Disease\n Occurrence,Symptom
0,UMLS:C0020538_hypertensive\n disease,3363,UMLS:C0008031_pain\n chest
1,,,UMLS:C0392680_shortness\n of breath
2,,,UMLS:C0012833_dizziness
3,,,UMLS:C0004093_asthenia
4,,,UMLS:C0085639_fall
...,...,...,...
1861,,,UMLS:C0425251_bedridden^UMLS:C0741453_bedridden
1862,,,UMLS:C0242453_prostatism
1863,UMLS:C0011127_decubitus\n ulcer,42,UMLS:C0232257_systolic murmur
1864,,,UMLS:C0871754_frail


In [3]:
# Step 1: Add a column to count the occurrences of "UMLS" in each row
df['UMLS_Count'] = df['Disease'].fillna('').str.count('UMLS')

# Step 2: Calculate the total occurrences of "UMLS" in the 'Disease' column
disease_umls_count = df['UMLS_Count'].sum()

# Print the total count of "UMLS"
print(f"Total occurrences of 'UMLS' in 'Disease' column: {disease_umls_count}")

# Step 3: Identify rows where a cell contains more than one occurrence of "UMLS"
rows_with_multiple_umls = df[df['UMLS_Count'] > 1]

# Print these rows
print("\nRows with more than one 'UMLS':")
display(rows_with_multiple_umls[['Disease','UMLS_Count']])

Total occurrences of 'UMLS' in 'Disease' column: 149

Rows with more than one 'UMLS':


,Disease,UMLS_Count
26,UMLS:C0011570_depression\n mental^UMLS:C00115...,2
47,UMLS:C0010054_coronary\n arteriosclerosis^UML...,2
295,UMLS:C0006826_malignant\n neoplasms^UMLS:C130...,2
305,UMLS:C0001175_acquiredimmuno-deficiency\n syn...,3
350,UMLS:C0036690_septicemia^UMLS:C0243026_systemi...,3
819,UMLS:C0376358_malignant\n neoplasm of prostat...,2
885,UMLS:C0006142_malignant\n neoplasm of breast^...,2
1294,UMLS:C0006840_candidiasis^UMLS:C0006849_oralca...,2
1357,UMLS:C0007102_malignant\n tumor of colon^UMLS...,2
1387,UMLS:C0014549_tonic-clonic\n epilepsy^UMLS:C0...,2


In [4]:
# Step 1: Clean and normalize the 'Disease' column
df['Disease'] = df['Disease'].str.replace('\n', ' ', regex=False).str.strip()

# Step 2: Filter rows containing valid 'UMLS:' entries
disease_rows = df[df['Disease'].notna() & df['Disease'].str.contains('UMLS:')]

# Step 3: Extract the first valid disease_id and d_name from each cell
disease_list = []  
seen_diseases = set()  

for row in disease_rows['Disease']:
    # Extract the first occurrence of a valid "UMLS:" entry
    entry = next((e.strip() for e in row.split('^') if e.startswith("UMLS:")), None)
    if entry and '_' in entry:  
        parts = entry.split('_', 1)  
        if len(parts) == 2:
            disease_id = parts[0].replace("UMLS:", "").strip()  
            # Replace underscores with spaces, then normalize spaces in d_name
            d_name = ' '.join(parts[1].replace('_', ' ').split()).lower()
            if (disease_id, d_name) not in seen_diseases:  
                disease_list.append({'disease_id': disease_id, 'd_name': d_name})
                seen_diseases.add((disease_id, d_name))
    else:
        # Log unmatched or invalid entries for debugging
        print(f"Unmatched or invalid entry: '{row}'")

# Step 4: Convert to DataFrame
if disease_list:
    diseases_df = pd.DataFrame(disease_list)

    # Step 5: Check for duplicate disease_id values
    duplicates = diseases_df[diseases_df['disease_id'].duplicated(keep=False)]  
    print("\nDuplicate disease_ids:")
    print(duplicates)

    # Step 6: Drop duplicate rows and keep the first occurrence
    diseases_df = diseases_df.drop_duplicates(subset='disease_id', keep='first')
    print("\nDuplicates dropped, DataFrame Updated.")

    # Final Output
    print(f"\nFinal diseases_df created with {len(diseases_df)} records.")
else:
    print("No valid disease entries found.")

# Display the resulting DataFrame
diseases_df



Duplicate disease_ids:
   disease_id               d_name
20   C0006826  malignant neoplasms
99   C0006826   malignantneoplasms

Duplicates dropped, DataFrame Updated.

Final diseases_df created with 133 records.


,disease_id,d_name
0,C0020538,hypertensive disease
1,C0011847,diabetes
2,C0011570,depression mental
3,C0010054,coronary arteriosclerosis
4,C0032285,pneumonia
...,...,...
129,C1258215,ileus
130,C0001511,adhesion
131,C0011253,delusion
132,C0233472,affect labile


In [5]:
# Extract d_names into a dictionary with index as key
if 'd_name' in diseases_df.columns:
    d_names_dict = diseases_df['d_name'].to_dict()  
    print("Disease Names:")
    pprint(d_names_dict)  
else:
    print("The 'd_name' column is missing!")

Disease Names:
{0: 'hypertensive disease',
 1: 'diabetes',
 2: 'depression mental',
 3: 'coronary arteriosclerosis',
 4: 'pneumonia',
 5: 'failure heart congestive',
 6: 'accidentcerebrovascular',
 7: 'asthma',
 8: 'myocardial infarction',
 9: 'hypercholesterolemia',
 10: 'infection',
 11: 'infection urinary tract',
 12: 'anemia',
 13: 'chronic obstructive airway disease',
 14: 'dementia',
 15: 'insufficiency renal',
 16: 'confusion',
 17: 'degenerativepolyarthritis',
 18: 'hypothyroidism',
 19: 'anxiety state',
 20: 'malignant neoplasms',
 21: 'acquiredimmuno-deficiency syndrome',
 22: 'cellulitis',
 23: 'gastroesophageal reflux disease',
 24: 'septicemia',
 25: 'deep vein thrombosis',
 26: 'dehydration',
 27: 'neoplasm',
 28: 'embolism pulmonary',
 29: 'epilepsy',
 30: 'cardiomyopathy',
 31: 'chronic kidney failure',
 32: 'carcinoma',
 33: 'hepatitis c',
 34: 'peripheral vascular disease',
 35: 'psychotic disorder',
 36: 'hyperlipidemia',
 37: 'bipolar disorder',
 38: 'obesity',
 39:

In [6]:
# Replace the URL below with the raw file URL for your text file
github_url = "https://raw.githubusercontent.com/Saurabh-Lakhanpal/symptoms-analyzer/refs/heads/main/resources/data/disease_description.txt"

# Fetch the file content from GitHub
response = requests.get(github_url)
content = response.text

# Parse the content to create a mapping of d_name to description
d_name_to_description = {}
for line in content.splitlines():
    if ": " in line:
        parts = line.split(": ", 1)
        d_name_key = parts[0].split(". ", 1)[1].strip().lower()
        description_value = parts[1].strip()
        d_name_to_description[d_name_key] = description_value

# Map descriptions to the DataFrame
diseases_df['description'] = diseases_df['d_name'].str.lower().map(d_name_to_description)

# Reset index of the updated DataFrame
diseases_df = diseases_df.reset_index(drop=True)

# Apply title case to columns 'd_name' and 'description'
diseases_df['d_name'] = diseases_df['d_name'].str.title()
diseases_df['description'] = diseases_df['description'].str.title()

# Display the updated DataFrame
print("\nUpdated diseases_df with reset index:")
diseases_df


Updated diseases_df with reset index:


,disease_id,d_name,description
0,C0020538,Hypertensive Disease,Chronic High Blood Pressure Increases Risk Of ...
1,C0011847,Diabetes,A Metabolic Disorder Causing Elevated Blood Su...
2,C0011570,Depression Mental,"A Condition Marked By Persistent Sadness, Low ..."
3,C0010054,Coronary Arteriosclerosis,Hardening Of Coronary Arteries Reduces Blood F...
4,C0032285,Pneumonia,"A Lung Infection That Causes Cough, Fever, And..."
...,...,...,...
128,C1258215,Ileus,Disruption Of The Normal Movement Of The Intes...
129,C0001511,Adhesion,Scar Tissue That Forms Between Organs Or Tissu...
130,C0011253,Delusion,"A Fixed False Belief, Often Associated With Me..."
131,C0233472,Affect Labile,"Rapid And Intense Emotional Changes, Commonly ..."


In [7]:
# Step 1: Check for duplicates in the 'disease_id' column
duplicates_in_disease_id = diseases_df[diseases_df['disease_id'].duplicated(keep=False)]  # Find all duplicate rows

# Count the number of unique and duplicate entries
total_duplicates_count = duplicates_in_disease_id.shape[0]
print(f"\nTotal duplicate entries in 'disease_id' column: {total_duplicates_count}")

# Step 2: Trim leading and trailing whitespaces from all string columns
diseases_df = diseases_df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
print("\nWhitespace trimming complete.")

# Step 3: Check for null values and display rows with NaN values
null_values_per_column = diseases_df.isnull().sum()
print("\nNull values in the DataFrame (per column):")
print(null_values_per_column)



Total duplicate entries in 'disease_id' column: 0

Whitespace trimming complete.

Null values in the DataFrame (per column):
disease_id     0
d_name         0
description    0
dtype: int64


In [8]:
# Step 1: Add a column to count the occurrences of "UMLS" in each row
df['UMLS_Count'] = df['Symptom'].fillna('').str.count('UMLS')

# Step 2: Calculate the total occurrences of "UMLS" in the 'Symptom' column
Symptom_umls_count = df['UMLS_Count'].sum()

# Print the total count of "UMLS"
print(f"Total occurrences of 'UMLS' in 'Symptom' column: {Symptom_umls_count}")

# Step 3: Identify rows where a cell contains more than one occurrence of "UMLS"
rows_with_multiple_umls = df[df['UMLS_Count'] > 1]

# Print these rows
print(f"\n{len(rows_with_multiple_umls)} Rows with more than one 'UMLS':")
display(rows_with_multiple_umls[['Symptom', 'UMLS_Count']])

Total occurrences of 'UMLS' in 'Symptom' column: 1907

41 Rows with more than one 'UMLS':


,Symptom,UMLS_Count
7,UMLS:C0038990_sweat^UMLS:C0700590_sweating\n ...,2
20,UMLS:C0038990_sweat^UMLS:C0700590_sweating\n ...,2
51,UMLS:C0038990_sweat^UMLS:C0700590_sweating\n ...,2
101,UMLS:C0038990_sweat^UMLS:C0700590_sweating inc...,2
119,UMLS:C0038990_sweat^UMLS:C0700590_sweating inc...,2
195,UMLS:C0425251_bedridden^UMLS:C0741453_bedridden,2
252,UMLS:C0013144_drowsiness^UMLS:C0234450_sleepy,2
264,UMLS:C0015672_fatigue^UMLS:C0557875_tired,2
298,UMLS:C0332601_cushingoid facies^UMLS:C0878661_...,2
317,UMLS:C0026827_muscle hypotonia^UMLS:C0241938_h...,2


In [9]:
# Step 1: Initialize a list to store extracted symptom_id and s_name pairs
symptom_list = []

# Step 2: Iterate through the 'symptom' column to process each cell
for row in df['Symptom']:
    if pd.notna(row):  
        entries = row.split('^')  
        for entry in entries:
            entry = entry.strip()  
            if entry.startswith("UMLS:") and '_' in entry:  
                parts = entry.split('_', 1)  
                if len(parts) == 2:
                    symptom_id = parts[0].replace("UMLS:", "").strip()  
                    s_name = ' '.join(parts[1].split()).lower()  
                    symptom_list.append({'symptom_id': symptom_id, 's_name': s_name})

# Step 3: Convert the extracted list into a DataFrame
if symptom_list:
    symptoms_df = pd.DataFrame(symptom_list)

    # Print the resulting DataFrame for preview
    print("\nsymptoms_df created successfully:")
    display(symptoms_df)
else:
    print("No valid symptom entries found in the column.")



symptoms_df created successfully:


,symptom_id,s_name
0,C0008031,pain chest
1,C0392680,shortness of breath
2,C0012833,dizziness
3,C0004093,asthenia
4,C0085639,fall
...,...,...
1902,C0741453,bedridden
1903,C0242453,prostatism
1904,C0232257,systolic murmur
1905,C0871754,frail


In [10]:
# Step 1: Remove rows with null values in 'symptom_id' or 's_name'
symptoms_df = symptoms_df.dropna(subset=['symptom_id', 's_name'])

# Step 2: Remove rows where 's_name' is empty or contains only whitespace
symptoms_df = symptoms_df[symptoms_df['s_name'].str.strip() != '']

# Step 3: Retain only rows with unique symptom_id
symptoms_df = symptoms_df.drop_duplicates(subset='symptom_id')

# Step 4: Convert s_name column to title case
symptoms_df['s_name'] = symptoms_df['s_name'].str.title()

# Step 5: Reset the index
symptoms_df = symptoms_df.reset_index(drop=True)

# Print the updated DataFrame
print("\nUpdated symptoms_df with unique symptom_id values, no nulls, s_name in title case, and reset index:")
symptoms_df


Updated symptoms_df with unique symptom_id values, no nulls, s_name in title case, and reset index:


,symptom_id,s_name
0,C0008031,Pain Chest
1,C0392680,Shortness Of Breath
2,C0012833,Dizziness
3,C0004093,Asthenia
4,C0085639,Fall
...,...,...
401,C0740844,Air Fluid Level
402,C0425491,Catching Breath
403,C0456091,Large-For-Dates Fetus
404,C0231441,Immobile


In [11]:
# Step 1: Create a copy of the original DataFrame
df_copy = df.copy()

# Step 2: Replace different representations of empty values with actual NA
df_copy['Disease'] = df_copy['Disease'].replace(['', 'None', 'nan'], pd.NA)

# Step 3: Fill empty rows with the last valid entry
last_valid_entry = None
for i, disease_entry in enumerate(df_copy['Disease']):
    if pd.notna(disease_entry):  # If the entry is valid, update last_valid_entry
        last_valid_entry = disease_entry
    else:  # If the entry is empty, use the last valid entry
        df_copy.at[i, 'Disease'] = last_valid_entry

# Step 4: Display the updated DataFrame
print("\nUpdated DataFrame (df_copy) with 'Disease' column filled:")
df_copy


Updated DataFrame (df_copy) with 'Disease' column filled:


,Disease,Count of Disease\n Occurrence,Symptom,UMLS_Count
0,UMLS:C0020538_hypertensive disease,3363,UMLS:C0008031_pain\n chest,1
1,UMLS:C0020538_hypertensive disease,,UMLS:C0392680_shortness\n of breath,1
2,UMLS:C0020538_hypertensive disease,,UMLS:C0012833_dizziness,1
3,UMLS:C0020538_hypertensive disease,,UMLS:C0004093_asthenia,1
4,UMLS:C0020538_hypertensive disease,,UMLS:C0085639_fall,1
...,...,...,...,...
1861,UMLS:C0233472_affect labile,,UMLS:C0425251_bedridden^UMLS:C0741453_bedridden,2
1862,UMLS:C0233472_affect labile,,UMLS:C0242453_prostatism,1
1863,UMLS:C0011127_decubitus ulcer,42,UMLS:C0232257_systolic murmur,1
1864,UMLS:C0011127_decubitus ulcer,,UMLS:C0871754_frail,1


In [12]:
# Step 1: Initialize lists for storing extracted data
data_list = []

# Create a set of valid symptom_ids from symptoms_df
valid_symptom_ids = set(symptoms_df['symptom_id'])

# Step 2: Iterate through each row of df_copy
for _, row in df_copy.iterrows():
    disease = row['Disease']
    symptom = row['Symptom']

    # Parse out disease_id and d_name
    if pd.notna(disease):
        disease_entry = next((e.strip() for e in disease.split('^') if e.startswith("UMLS:")), None)
        if disease_entry and '_' in disease_entry:
            disease_parts = disease_entry.split('_', 1)
            if len(disease_parts) == 2:
                disease_id = disease_parts[0].replace("UMLS:", "").strip()
                d_name = ' '.join(disease_parts[1].replace('_', ' ').split()).lower().title()
    else:
        disease_id = None
        d_name = None

    # Parse out symptom_id and s_name
    if pd.notna(symptom):
        symptom_entries = symptom.split('^')
        for sym_entry in symptom_entries:
            sym_entry = sym_entry.strip()
            if sym_entry.startswith("UMLS:") and '_' in sym_entry:
                sym_parts = sym_entry.split('_', 1)
                if len(sym_parts) == 2:
                    symptom_id = sym_parts[0].replace("UMLS:", "").strip()
                    s_name = ' '.join(sym_parts[1].replace('_', ' ').split()).lower().title()

                    # Only append if symptom_id is valid
                    if symptom_id in valid_symptom_ids:
                        data_list.append({
                            'disease_id': disease_id,
                            'd_name': d_name,
                            'symptom_id': symptom_id,
                            's_name': s_name
                        })
    else:
        # If no symptoms are present, add a blank row
        data_list.append({
            'disease_id': disease_id,
            'd_name': d_name,
            'symptom_id': None,
            's_name': None
        })

# Step 3: Convert the list into a DataFrame
disease_symptom_df = pd.DataFrame(data_list)

# Display the resulting DataFrame
print("\nFinal disease_symptom_df created:")
disease_symptom_df



Final disease_symptom_df created:


,disease_id,d_name,symptom_id,s_name
0,C0020538,Hypertensive Disease,C0008031,Pain Chest
1,C0020538,Hypertensive Disease,C0392680,Shortness Of Breath
2,C0020538,Hypertensive Disease,C0012833,Dizziness
3,C0020538,Hypertensive Disease,C0004093,Asthenia
4,C0020538,Hypertensive Disease,C0085639,Fall
...,...,...,...,...
1901,C0233472,Affect Labile,C0741453,Bedridden
1902,C0233472,Affect Labile,C0242453,Prostatism
1903,C0011127,Decubitus Ulcer,C0232257,Systolic Murmur
1904,C0011127,Decubitus Ulcer,C0871754,Frail


In [13]:
# Exporting the disease_symptom_df DataFrame to a CSV file
disease_symptom_df.to_csv("../resources/disease_symptom_data.csv", index=False)

print("DataFrame 'disease_symptom_df' exported successfully!")


DataFrame 'disease_symptom_df' exported successfully!


In [14]:
import psycopg2
from sqlalchemy import create_engine
from psycopg2 import sql


# Define initial and target database parameters
initial_db_params = {
    'dbname': 'postgres',  
    'user': 'postgres',
    'password': 'postgres',
    'host': 'localhost',
    'port': '5432'
}

db_params = {
    'dbname': 'disease_symptom_db',  
    'user': 'postgres',
    'password': 'postgres',
    'host': 'localhost',
    'port': '5432'
}

# Functions for database operations
def terminate_sessions(cursor, dbname):
    cursor.execute(sql.SQL("""
        SELECT pg_terminate_backend(pid)
        FROM pg_stat_activity
        WHERE datname = %s AND pid <> pg_backend_pid();
    """), [dbname])

def drop_database(cursor, dbname):
    cursor.execute("SELECT pg_terminate_backend(pg_stat_activity.pid) FROM pg_stat_activity WHERE pg_stat_activity.datname = %s AND pid <> pg_backend_pid();", [dbname])
    cursor.execute(sql.SQL("DROP DATABASE IF EXISTS {}").format(sql.Identifier(dbname)))

def create_database(cursor, dbname):
    cursor.execute(sql.SQL("CREATE DATABASE {}").format(sql.Identifier(dbname)))

def execute_sql_file(cursor, sql_file_path):
    with open(sql_file_path, 'r') as file:
        sql_commands = file.read()
    cursor.execute(sql.SQL(sql_commands))

sql_file_path = 'schema.sql'

# Connect to the initial database and create the target database
try:
    # Connect to the initial database (postgres)
    connection = psycopg2.connect(**initial_db_params)
    connection.autocommit = True
    cursor = connection.cursor()

    # Drop the target database if it exists
    drop_database(cursor, db_params['dbname'])
    print(f"Database {db_params['dbname']} dropped successfully.")

    # Create the target database
    create_database(cursor, db_params['dbname'])
    print(f"Database {db_params['dbname']} created successfully.")

    # Close the initial connection
    cursor.close()
    connection.close()

    # Connect to the newly created target database (stocks_dashboard_db)
    connection = psycopg2.connect(**db_params)
    cursor = connection.cursor()

    # Execute the SQL schema file to set up the database
    execute_sql_file(cursor, sql_file_path)
    connection.commit()
    print("SQL schema file executed successfully.")

except Exception as e:
    print(f"An error occurred: {e}")
    if connection:
        connection.rollback()

finally:
    if cursor:
        cursor.close()
    if connection:
        connection.close()
        
# ==============================================================================================================================================

# Function to upload DataFrame to PostgreSQL table
def upload_df_to_table(connection_string, table_name, df):
    engine = create_engine(connection_string)
    df.to_sql(table_name, con=engine, if_exists='append', index=False)
    return len(df)

# Ensure column names in diseases_df and symptoms_df match the table schema
diseases_df.columns = ['disease_id', 'd_name', 'description']
symptoms_df.columns = ['symptom_id', 's_name']
disease_symptom_df.columns = ['disease_id', 'd_name', 'symptom_id', 's_name']

# PostgreSQL connection string
connection_string = f"postgresql://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['dbname']}"

# Upload the DataFrames to PostgreSQL tables
try:
    # Upload diseases_df to diseases_tb
    diseases_rows = upload_df_to_table(connection_string, 'diseases_tb', diseases_df)
    print(f"Data for diseases_tb uploaded successfully. Total rows: {diseases_rows}")
    
    # Upload symptoms_df to symptoms_tb
    symptoms_rows = upload_df_to_table(connection_string, 'symptoms_tb', symptoms_df)
    print(f"Data for symptoms_tb uploaded successfully. Total rows: {symptoms_rows}")
    
    # Upload disease_symptom_df to disease_symptom_tb
    disease_symptom_rows = upload_df_to_table(connection_string, 'disease_symptom_tb', disease_symptom_df)
    print(f"Data for disease_symptom_tb uploaded successfully. Total rows: {disease_symptom_rows}")

except Exception as e:
    print(f"An error occurred while uploading data: {e}")


Database disease_symptom_db dropped successfully.
Database disease_symptom_db created successfully.
SQL schema file executed successfully.
Data for diseases_tb uploaded successfully. Total rows: 133
Data for symptoms_tb uploaded successfully. Total rows: 406
Data for disease_symptom_tb uploaded successfully. Total rows: 1906
